# Transcript A from Lecture, October 26, 2021


In [ ]:
import sys

########################################
# Change the string in the line below! #
########################################
sys.path.append("/Users/gilbert/Documents/CS111-2021-fall/Python") 

import os
import time
import math
import numpy as np
import numpy.linalg as npla
import scipy
from scipy import linalg as spla
import scipy.sparse
import scipy.sparse.linalg
from scipy import integrate
import networkx as nx
import cs111

##########################################################
# If this import for matplotlib doesn't work, try saying #
#   conda install -c conda-forge ipympl                  #
# at a shell prompt on your computer                     #
##########################################################
import matplotlib
%matplotlib ipympl

import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import axes3d




np.set_printoptions(precision = 4)

# SVD theorems

In [ ]:
A = np.random.rand(8,5)
print('shape of A:', A.shape)
print()

U,sigma,Vt = spla.svd(A)

print('shape of U:', U.shape)
print('sigma:', sigma)
print('shape of Vt:', Vt.shape)


<h3>Theorem 1. The rank of $A$ is the number of nonzero singular values.</h3>

In our example, sure enough, the matrix $A$ has rank 5.

In [ ]:
print('singular values:', sigma)
print('rank(A):', npla.matrix_rank(A))

In floating-point arithmetic, what counts as "nonzero" can be a judgement call. To illustrate this, we form a new 8-by-5 matrix $B$ whose first 4 columns are the same as $A$ and whose 5th column is the sum of the first 4 columns. 

In [ ]:
B = A.copy()
B[:,4] = A @ [1, 1, 1, 1, 0]
# can you see why this replaces column 4 of B with the sum of the first 4 columns of A?

print('B:\n', B)
print()
print('B @ [1, 1, 1, 1, -1] should be the zero vector:\n\n', B @ [1, 1, 1, 1, -1])
print()
print('norm of B @ [1, 1, 1, 1, -1]:', npla.norm(B @ [1, 1, 1, 1, -1]))

Mathematically speaking, $B$ should have rank 4 because one column is linearly dependent on the other four columns. In floating-point arithmetic, though, round-off error makes the last column not quite exactly equal to the sum of the others. The exact rank of this perturbed matrix is 5, and sure enough all of its computed singular values are nonzero. 

In [ ]:
UB, sigmaB, VtB = spla.svd(B)
print('singular values of B:', sigmaB)

However, because only 4 of the singular values are significantly larger than machine epsilon, we (and numpy) say that $B$ has "numerical rank" 4.

In [ ]:
npla.matrix_rank(B)

We defined the 2-norm of a matrix a while ago, 
but we never talked about algorithms to compute it.
It turns out that the SVD gives us an algorithm.

<h3>Theorem 2. The 2-norm $||A||_2$ is equal to the largest singular value $\sigma_0$.</h3>

This is because the norm of $A$ is the largest stretch it applies to any vector, 

$$||A||_2 = \max_x (||Ax||_2/||x||_2).$$

We saw that $A$ maps the unit sphere to an ellipsoid, and the largest stretch happens at the longest axis of the ellipsoid, whose length is $\sigma_0$. The actual vectors are $x=v_0$ and $Ax = \sigma_0 u_0$.

In [ ]:
v0 = V[:,0]
u0 = U[:,0]

print('singular values of A:', sigma)
print('2-norm of A:', npla.norm(A,2))
print()

print('v_0:', v0)
print('u_0:', u0)
print('A @ v_0:', A@v0)
print()
print('norm(A @ v_0) / norm(v_0) :', npla.norm(A@v0, 2) / npla.norm(v0, 2))

The SVD also gives us a way to compute the condition number of $A$ in the 2-norm, $\kappa_2(A)$.
Recall the definition

$$\kappa(A) = \max(\mbox{stretch}) / \min(\mbox{stretch}),$$

the ratio of the maximum amount any vector is stretched by $A$ to the minimum amount 
any vector is stretched by $A$. 
(The minimum is 0 if $A$ is square and singular, 
or more generally if the rank of $A$ is less than the number of columns,
and in that case the condition number is infinite.
If $A$ is square and nonsingular, the condition number is $||A|| / ||A^{-1}||.$)

From the discussion of $||A||_2$ above, it is clear that

<h3>Theorem 3. The 2-norm condition number  $\kappa_2(A)$ is equal to the ratio of the largest and smallest singular values,</h3>

$$\kappa_2(A) = \sigma_0 / \sigma_{\min(m,n)-1}$$

In [ ]:
print('ratio of extreme singular values: ', sigma[0]/sigma[-1])
#print('2-norm condition number of matrix:', npla.cond(A,2))


In [ ]:
# Unfortunately numpy's condition number routine doesn't work for non-square matrices!

npla.cond(A,2)

The _Frobenius norm_ of a matrix is the square root of the sum of the squares of all
of its elements. It's as if we took the whole $m$-by-$n$ matrix as a vector in $mn$-dimensional space and computed its Euclidean length. This is an easy norm to compute, so we often use it to compare two matrices with each other. It turns out the SVD gives the Frobenius norm too.

<h3>Theorem 4. The Frobenius norm $||A||_F$ is equal to $(\sum_i \sigma_i^2)^{1/2}$.</h3>

Let's check it out.

In [ ]:
sumsig = 0
for i in range(len(sigma)):
    sumsig += sigma[i]**2
print('sqrt(sum(singular values squared)):', np.sqrt(sumsig))

nrows, ncols = A.shape
sumsqA = 0
for i in range(nrows):
    for j in range(ncols):
        sumsqA += A[i,j]**2
print('sqrt(sum(matrix elements squared)):', np.sqrt(sumsqA))

print('Frobenius norm of matrix:          ', npla.norm(A,'fro'))


The determinant of a matrix is important in matrix theory, 
but is hardly ever computed in numerical linear algebra where matrix norms and
condition numbers are more useful.
However, the SVD does give a way to compute the determinant of a (square) matrix,
up to its sign.

<h3>Theorem 5. The determinant of a square matrix is $\pm$ the product of its singular values, $\prod_i \sigma_i$.</h3>

In [ ]:
Asquare = A[:5,:]
print('shape of square matrix:', Asquare.shape)
UAs, sigmaAs, VtAs = npla.svd(Asquare)

prodsig = 1
for i in range(len(sigmaAs)):
    prodsig *= sigmaAs[i]
print('product of singular values:', prodsig)

print('determinant of matrix:     ', npla.det(Asquare))


# Sum of rank-1 matrices, and low-rank approximation

Our final theorem is just a simple way to rewrite the SVD equation $A=USV^T$.
Recall that $u_i$ is column $i$ of $U$ and $v_i$ is column $i$ of $V$, so $v_i^T$ is row $i$ of $V^T$.
It is straightforward to check algebraically that

<h3>Theorem 6. Matrix $A$ is the sum of rank-1 matrices: $A = \sum_{i=0}^{\min(m,n)}\sigma_i u_i v_i^T$ </h3>

Each term in the sum is a scalar multiple of the outer product $u_i v_i^T$,
which is an $m$-by-$n$ matrix whose rank is 1;
it's essentially the multiplication table of the elements of $u_i$ (as rows) times
the elements of $v_i$ (as columns).
(Indeed, every matrix product $AB$ can be written as a sum of rank-1 matrices, 
each of which is the outer product of a column of $A$ and a row of $B$.)

Though it's just a humble algebraic identity, this last theorem actually motivates
the greatest applications of SVD in data analysis. 
We'll see that in the next section, but first let's just check the theorem numerically 
for our example matrix.

We ought to be able to compute $u_i v_i^T$ as U[:,i] @ V[:,i].T in numpy.
Unfortunately numpy is broken here -- it gets confused because it can't tell that U[:,i] is a column vector and V[:,i].T is a row vector, and it does the wrong thing.
(Matlab on the other hand gets this right.)
In numpy, we have to use the np.outer() function to compute the outer product of two vectors,
as we do below.

In [ ]:
Asum = np.zeros(A.shape)
for i in range(len(sigma)):
    Asum += sigma[i] * np.outer(U[:,i], V[:,i])

print('norm of difference between Asum and A:',  npla.norm(Asum - A))

What happens if we truncate the sum in Theorem 6 (above) after some number $k<\min(m,n)$ of terms? Let us define

$$A_k = \sum_{i=0}^{k-1}\sigma_i u_i v_i^T,$$

with $A_{\min(m,n)} = A$.
We can think of each rank-1 term $\sigma_i u_i v_i^T$ as adding some "weight" to the matrix, in an informal sense. The terms are added in order of decreasing weight, because $\sigma_0 \ge \sigma_1 \ge \cdots$. If the first singular values are much larger than the later ones, we might hope that $A_k$ would be a good approximation to $A$ for small values of $k$. This turns out to be true in a very strong sense: $A_k$ is the _best_ possible rank-$k$ approximation to $A$, as measured in either the 2-norm or the Frobenius norm.

<h3> Theorem 7. Among all $m$-by-$n$ matrices $B_k$ that have rank $k$, 
the minimum possible value of $||A-B_k||_2$ is attained when $B_k=A_k$ as defined above.
That value is $||A-A_k||_2 = \sigma_{k}$.</h3>

<h3> Theorem 8. Among all $m$-by-$n$ matrices $B_k$ that have rank $k$, 
the minimum possible value of $||A-B_k||_F$ is attained when $B_k=A_k$.
That value is $||A-A_k||_F = (\sum_{i\ge k}\sigma_{k}^2)^{1/2}$. </h3>

For the case $k=0$, we get the results of Theorems 2 and 4 above. 
For $k\ge \mbox{rank}(A)$, we get $A=A_k$.

We illustrate Theorem 7 with our sample 8-by-5 matrix.

In [ ]:
nrows, ncols = A.shape
print('shape of A:', (nrows, ncols))

U,sigma,Vt = spla.svd(A)

print('singular values:', sigma)
print('rank(A):', npla.matrix_rank(A))
print()

Ak = np.zeros(A.shape)
for k in range(len(sigma)):
    print('rank', npla.matrix_rank(Ak), 'approximation: 2-norm(A%d - A) =' % k, npla.norm(Ak-A,2))
    Ak += sigma[k] * np.outer(U[:,k], Vt[k,:])
print('rank', npla.matrix_rank(Ak), 'approximation: 2-norm(A%d - A) =' % len(sigma), npla.norm(Ak-A,2))
